In [1]:
import copy
import logging
import sys

from pathlib import Path
import opt_einsum as oe
import numpy as np
from matplotlib import pyplot as plt
import pyscf

from dft2cc.dft2cc import DFT2CC
from dft2cc.utils.mol import Mol, PREDICT_MOLECULAR


name_mol = "HCHHH"
level = 4
distance = -0.25
molecular = copy.deepcopy(Mol[name_mol])
molecular[0][1] = distance + molecular[0][1]
save_data = {}

# set the target of logger to stderr
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)

handler = logging.StreamHandler(sys.stderr)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

path_dir = Path("../tmp")

dft2cc = DFT2CC(
    molecular,
    path=path_dir / f"{distance:.4f}",
    logger=logger,
    level=level,
    basis="cc-pcvdz",
    noisy_print=False,
    if_basis_str=True,
)

ccsd_dm1_r = np.load(path_dir / f"{distance:.4f}" / "ccsd_dm1_r.npy")

/home/dhem/anaconda3/envs/pyscf/lib/python3.11/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [18]:
mat_a = np.zeros((len(ccsd_dm1_r), dft2cc.norb**2))
for i in range(dft2cc.norb):
    for j in range(dft2cc.norb):
        mat_a[:, i * dft2cc.norb + j] = dft2cc.ao_0[:, i] * dft2cc.ao_0[:, j]

res = np.linalg.lstsq(mat_a, ccsd_dm1_r, rcond=None)

dm1_fitted = res[0].reshape((dft2cc.norb, dft2cc.norb))
residuals = res[1]

dm1_fitted_r = oe.contract(
    "uv,gu,gv->g",
    dm1_fitted,
    dft2cc.ao_0,
    dft2cc.ao_0,
    optimize="auto",
)

print(np.linalg.norm(dm1_fitted_r - ccsd_dm1_r))

# ccsd_error_dm1_r = oe.contract(
#     "uv,gu,gv->g",
#     ccsd_dm1,
#     dft2cc.ao_0,
#     dft2cc.ao_0,
#     optimize="auto",
# )

# print(np.linalg.norm(ccsd_dm1_r - ccsd_error_dm1_r, ord=1))
# print(
#     np.sum(ccsd_dm1_r * dft2cc.grids.weights)
#     - np.sum(ccsd_error_dm1_r * dft2cc.grids.weights)
# )